In [1]:
import mne.time_frequency

from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np

In [2]:

# manually set this for now since mac is different from windows, make more robust later
# HOME = '/Users/jinjiang-macair/Library/CloudStorage'
# os.path.expanduser("~") SHOULD be '/Users/jinjiang-macair/'
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")


C:\Users\Jakda\anaconda3\envs\ieeg\Lib\site-packages\bids\layout\layout.py:515: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([WindowsPath('C:/Users/Jakda/Box/CoganLab/BIDS-1.0_GlobalLocal/BIDS/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


In [ ]:
print(subjects)
for sub in subjects:
    # if sub != "D0022":
    #     continue
    # Load the data
    filt = raw_from_layout(layout, subject=sub,
                           extension='.edf', preload=False)
    print(filt)

    ## Crop raw data to minimize processing time
    good = crop_empty_data(filt)

    # good.drop_channels(good.info['bads'])
    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    good.drop_channels(good.info['bads'])
    # good.info['bads'] += channel_outlier_marker(good, 4, 2)
    # good.drop_channels(good.info['bads'])
    good.load_data()

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    # good.plot()

    ## epoching and trial outlier removal

    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # This part will change when I fix the events
    for epoch, t, name in zip(
            ("Stimulus", "Stimulus",  "Response"),
            ((-0.5, 0), (0, 1), (-0.5, 0.5)),
            ("base", "stim", "resp")):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=-2, decim=int(
            good.info['sfreq'] / 100))
        crop_pad(spec, "0.5s")
        if name == "base":
            base = spec.copy()
            continue
        spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
        spec_a._data = np.log10(spec_a._data) * 20
        fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
        spec_a.info['subject_info']['files'] = tuple(fnames)
        spec_a.info['bads'] = good.info['bads']
        filename = os.path.join(save_dir, f'{name}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
        spec_a.save(os.path.join(save_dir, f'{name}-avg.fif'), overwrite=True)

['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071']
Extracting EDF parameters from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_events.tsv.
Reading channel info from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_channels.tsv.
Reading electrode coords from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Jakda\git\GlobalLocal\IEEG_Pipelines\ieeg\io.py:100: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\Jakda\git\GlobalLocal\IEEG_Pipelines\ieeg\io.py:100: RuntimeWarning: participants.tsv file not found for C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_events.tsv.
Reading channel info from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_channels.tsv.
Reading electrode coords from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Jakda\git\GlobalLocal\IEEG_Pipelines\ieeg\io.py:100: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\Jakda\git\GlobalLocal\IEEG_Pipelines\ieeg\io.py:100: RuntimeWarning: participants.tsv file not found for C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_events.tsv.
Reading channel info from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_channels.tsv.
Reading electrode coords from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\Jakda\Box\CoganLab\BIDS-1.0_GlobalLocal\BIDS\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:   59.1s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:  5.0min


Not setting metadata


[Parallel(n_jobs=-2)]: Done 177 out of 177 | elapsed:  6.0min finished


Used Annotations descriptions: ['Response', 'Stimulus']
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 448 events and 4097 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   39.5s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:  3.5min
